# Model 01. For running with cells vs marker matrices.

TODO: Adjust tensor sizes (maybe zero-padding, reducing, or augmenting?)

To run with tensorboard.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install torch torchvision

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
## From Hu et al. (2020)
import pickle
import pandas as pd
import numpy as np
from numpy.random import seed; seed(111)
import random
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from IPython.display import Image

## 0. Import
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F

In [ ]:
class AdDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.x = os.listdir(data_dir)
        self.y = []  # Initialize an empty list to store class labels
        self.dimensions = []  # Initialize an empty list to store dimensions

        for file_name in self.x:
          # Extract the part of the file name containing the label
          label_part = file_name.split(" ")[2]
          # Extract the letter from the label part
          y_label = label_part.split("_")[0]
          #print(y_label)
          # Check if the class label is "C" and assign 0, else assign 1
          if y_label == "C":
              self.y.append(0)
              #print("C")
          else:
              self.y.append(1)
              #print("P")

          file_path = os.path.join(self.data_dir, file_name)
          data = pd.read_csv(file_path, sep=",", header=0).values
          # if y_label=="C":

    def __len__(self):
        # Size of the whole data set
        return len(self.x)

    def __getitem__(self, idx):

        file_path = os.path.join(self.data_dir, self.x[idx])
        data = pd.read_csv(file_path, sep="\t", header=None).values

        # Select 10,000 random rows from the data
        random_indices = random.sample(range(len(data)), 10000)
        data = data[random_indices]
        label = self.y[idx] # Retrieve corresponding labels

        data = torch.from_numpy(data)
        # print("Input shape should be", shape)

        return data, label

    # def get_dimensions(self):
    #     return self.dimensions

NameError: ignored

In [ ]:
data_dir_train = "/content/drive/MyDrive/colabData/ST1_20181025/train" # 290
train_dataset = AdDataset(data_dir_train)

data_dir_val = "/content/drive/MyDrive/colabData/ST1_20181025/validate"
val_dataset = AdDataset(data_dir_val)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=4, shuffle=True)


(44267, 35)
(74892, 35)
(10563, 35)
(13414, 35)
(178650, 35)
(138863, 35)
(115418, 35)
(167620, 35)
(294848, 35)
(145652, 35)
(129237, 35)
(177182, 35)
(133125, 35)
(72079, 35)
(20434, 35)
(106058, 35)
(298104, 35)
(180851, 35)
(2655, 35)
(309058, 35)
(274629, 35)


In [ ]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [ ]:
class ClassificationBase(nn.Module):
    def training_step(self, batch):
        #inputs, classes = batch
        images, targets = batch
        images = images.type(torch.FloatTensor) # Uncomment for BreastCancer ClassfierBase class
        #images = torch.reshape(images.type(torch.DoubleTensor), (len(images), 1))
        targets = torch.reshape(targets.type(torch.FloatTensor), (len(targets), 1))
        out = self(images)
        loss = F.binary_cross_entropy(out, targets)
        return loss

    def validation_step(self, batch):
        images, targets = batch
        images = images.type(torch.FloatTensor) # Uncomment for BreastCancer ClassfierBase class
        #images = torch.reshape(images.type(torch.DoubleTensor), (len(images), 1))
        #print(images)
        targets = torch.reshape(targets.type(torch.FloatTensor), (len(targets), 1))
        #print(targets)
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda') #REQUIRES CHANGING THE TORCH.FLOATTENSOR TO TORCH.CUDA.FLOATTENSOR
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

device(type='cpu')

In [ ]:
train_dl = DeviceDataLoader(train_loader, device)
val_dl = DeviceDataLoader(val_loader, device)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_loader))

    #writer = SummaryWriter()  # Create a SummaryWriter instance

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []  # learning rate
        step = 0  # Initialize the step counter
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Write the training loss to TensorBoard with unique step for each batch
            writer.add_scalar('Training Batch Loss', loss, step)
            step += 1  # Increment the step counter

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Write the training loss and learning rate to TensorBoard
        writer.add_scalar('Training Loss', torch.stack(train_losses).mean().item(), epoch)
        writer.add_scalar('Learning Rate', lrs[-1], epoch)

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

    return history

def plot_scores(history):
    scores = [x['val_score'] for x in history]
    plt.plot(scores, '-x')
    plt.xlabel('epoch')
    plt.ylabel('score')
    plt.title('F1 score vs. No. of epochs')
    plt.show()
    #plt.savefig("DNN_scores_no_augmentation")

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
    plt.show()
    #plt.savefig("DNN_losses_no_augmentation")

def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.')
    plt.show()
    #plt.savefig("DNN_lrs_no_augmentation")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
class DNN2(ClassificationBase):
    def __init__(self, input_channels, flat_shape):
        super().__init__()
        #channels, height, width = input_shape
        ## First convolutional layer
        # "Uses three filters to scan each row of the CyTOF data. This layer extracts relevant information from the cell marker profile of each cell." Is this grid AxBxC? Fix in inputShape[X]. Filter size = 1 x B
        # We want to measure C markers.
        # How many output markers
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=1, kernel_size=(2,2)) #(1,A)? - THE NUMBER OF NODES IN THE INPUT VECTOR. OR JUST KERNEL SIZE = 3?
        self.bn1 = torch.nn.BatchNorm2d(1)
        self.act1 = nn.ReLU()

        ## Second convolution layer
        # The second convolution layer uses three filters to scan each row of the first layer's output. Each filter combines information from the first layer for each cell.
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(2,2))
        self.bn2 = torch.nn.BatchNorm2d(1)
        self.act2 = nn.ReLU()

        ## Pooling layer
        # "The pooling layers averages the outputs of the second convolution layer. The purpose is to aggregate the cell level information into sample-level information.""

        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        #self.pool1 = nn.AvgPool2d(kernel_size=3, stride=2) #1,1 would not change anything right?
        self.flat = nn.Flatten()

        ## Dense layer
        # "The dense layer further extracts information from the pooling layer."
        self.fc1 = nn.Linear(in_features=flat_shape, out_features=2048) #In features = , out features
        #self.fc = nn.Linear(in_features=flat_shape, out_features=1)
        # Better to make biggest jump here or reduce slowly?
        self.bn3 = torch.nn.BatchNorm1d(2048)
        self.act3 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.1)

        # "The dense layer further extracts information from the pooling layer."
        self.fc2 = nn.Linear(in_features=2048, out_features=1028) #In features = , out features
        self.bn4 = torch.nn.BatchNorm1d(1028)
        self.act4 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.1)

        # "The dense layer further extracts information from the pooling layer."
        self.fc3 = nn.Linear(in_features=1028, out_features=512) #In features = , out features
        self.bn5 = torch.nn.BatchNorm1d(512)
        self.act5 = nn.ReLU()
        self.do3 = nn.Dropout(p=0.1)

        # "The dense layer further extracts information from the pooling layer."
        self.fc4 = nn.Linear(in_features=512, out_features=256) #In features = , out features
        self.bn6 = torch.nn.BatchNorm1d(256)
        self.act6 = nn.ReLU()
        self.do4 = nn.Dropout(p=0.1)

        ## Output layer
        # "The output layer uses logistic regression to report the probability of CMV infection for each sample."
        self.fc5 = nn.Linear(in_features=256, out_features=1)
        #self.bn3 = nn.BatchNorm1d(1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Reshape input size
        #print("Input dimensions", x.shape)
        #x = x.to(torch.float32)
        x = x.float()
        x = x.unsqueeze(1)
        #print("Input dimensions", x.shape)
        out = self.conv1(x)
        ##print("Input dimensions conv1", out.shape)
        out = self.bn1(out)
        ##print("Input dimensions bn1", out.shape)
        out = self.act1(out)
        #print("Input dimensions act1", out.shape)

        out = self.conv2(out)
        #print("Input dimensions conv2", out.shape)
        out = self.bn2(out)
        #print("Input dimensions bn2", out.shape)
        out = self.act2(out)
        #print("Input dimensions act2", out.shape)

        out = self.pool1(out)
        #print("Input dimensions pool1", out.shape)
        out = self.flat(out)
        #print("Input dimensions flat", out.shape)

        #out = out.reshape(-1, input_size)
        #print("Out dimensions", out.shape)
        out = self.fc1(out)
        #out = self.fc(out)
        #print("Input dimensions flat", out.shape)
        #flat_shape = out.shape[1]
        out = self.act3(out)
        out = self.bn3(out)
        out = self.do1(out)
        #print("Input dimensions do1", out.shape)

        out = self.fc2(out)
        #print("Input dimensions fc2", out.shape)
        out = self.act4(out)
        out = self.bn4(out)
        out = self.do2(out)

        out = self.fc3(out)
        #print("Input dimensions fc3", out.shape)
        out = self.act5(out)
        #out = self.sigmoid(out)
        out = self.bn5(out)
        #print("Input dimensions bn3", out.shape)

        out = self.fc4(out)
        out = self.act6(out)
        #print("Input dimensions fn4", out.shape)
        out = self.bn6(out)
        out = self.do4(out)


        out = self.fc5(out)
        #print("Input dimensions fn5", out.shape)
        out = self.sigmoid(out)


        return out


In [ ]:
#input=[1, 200, 200]
# input_size=200*200
# model = to_device(DNN1(input_shape=input), device) #, flat_shape=9801), device) #DONT WANT THIS TO BE AN INPUT!
# epochs = 100
# max_lr = 0.01
opt_func = torch.optim.Adam

# Run classifier
# history = [evaluate(model, val_dl)]
# history

# Train
# start_time = time.time()
# history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, opt_func=opt_func)
# train_time = time.time() - start_time
# # total_train_time = time.time() - start_time
# print("Total training time =", total_train_time)
# writer.flush
# writer.close()

from itertools import product

# Define the hyperparameter values to explore
epochs_values = [1, 2, 3]
max_lr_values = [0.01, 0.001]

best_score = 0.0
best_epochs = 0
best_max_lr = 0.0

product(epochs_values, max_lr_values)


In [ ]:
for epochs, max_lr in product(epochs_values, max_lr_values):

    # Create a unique tag for each run based on the hyperparameters
    tag = f"epochs_{epochs}_max_lr_{max_lr}"

    # Create a SummaryWriter instance for each run
    writer = SummaryWriter(log_dir=f"runs/{tag}")

    # Create a new instance of the model for each combination of hyperparameters
    model = to_device(DNN2(input_channels=1, flat_shape=9801), device)
    #model = to_device(DNN(), device)

    # Train the model and evaluate its performance
    history = [evaluate(model, val_dl)]
    history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, opt_func=opt_func)

    # Calculate the validation score
    final_score = history[-1]['val_score']

    # Check if the current combination is the best
    if final_score > best_score:
        print("Better parameters found, updating.")
        best_score = final_score
        best_epochs = epochs
        best_max_lr = max_lr

    # Print the validation score for the current combination
    print(f"Epochs: {epochs}, Max LR: {max_lr}, Validation Score: {final_score}")

    # Close the writer for each run
    writer.close()


# Print the best hyperparameters
print("Best Hyperparameters:")
print("Epochs:", best_epochs)
print("Max LR:", best_max_lr)

If above works, add tensor board to see output.
Then change architecture and num epochs

In [ ]:
!yes|tensorboard dev upload --logdir /content/runs/ --name "DNN2() with 25x25 " --description "CNN with 30/70 stratified split, kernel conv1"
